#### Setup

In [ ]:
from google.colab import drive
import sys

In [ ]:
drive.mount('/content/drive')
sys.path.append('/content/drive/Shareddrives/CMPE260/')

Mounted at /content/drive


In [ ]:
%run /content/drive/Shareddrives/CMPE260/rec_implementation_w_features.ipynb

cpu


#### Training

In [ ]:
train_env = Environment_w_Features(train_matrix)

hits, dcgs = [], []
hits_all, dcgs_all = [], []
step, best_step = 0, 0
step, best_step, best_step_all = 0, 0, 0

users = np.random.permutation(apt_users)

In [ ]:
for u in tqdm.tqdm(users):
    print('user =', u)
    user, country, age, memory = train_env.reset(u)

    for t in range(int(train_matrix[u].sum())):
      try:
        # given current state, get action using actor network
        action_emb = policy_network(state_representation(country, age, memory))
        action = policy_network.get_action(user, torch.tensor(train_env.memory[user.detach().cpu().numpy().astype(int), :]), state_representation, action_emb,
            torch.tensor([item for item in train_env.available_books if item not in train_env.viewed_books]).long()
        )
      except Exception as e:
        print(e)

      # update replay buffer
      user, memory, reward, done = train_env.step(action, action_emb, replay_buffer)

      # update weights of action and critic network
      if len(replay_buffer) > params['batch_size']:
          update(params['batch_size'], params['gamma'], params['min_value'], params['max_value'], params['soft_tau'])

      # monitor training process by evaluating on only one user for efficiency
      if step % 100 == 0 and step > 0:
          hit, dcg = evaluation(policy_network, state_representation, train_env.memory, test_loader)

          hits.append(hit)
          dcgs.append(dcg)

          if np.mean(np.array([hit, dcg]) - np.array([hits[best_step], dcgs[best_step]])) >= 0:
              best_step = step // 100
              torch.save(policy_network.state_dict(), params['log_w_features_dir'] + 'policy_net.pth')
              torch.save(value_network.state_dict(), params['log_w_features_dir'] + 'value_net.pth')
              torch.save(state_representation.state_dict(), params['log_w_features_dir'] + 'state_repr.pth')

      # evaluate for best policy after every 1000 steps on the whole test set
      if step % 1000 == 0 and step > 0:
          hit, dcg = evaluation(policy_network, state_representation, train_env.memory, full_loader)

          hits_all.append(hit)
          dcgs_all.append(dcg)

          if np.mean(np.array([hit, dcg]) - np.array([hits_all[best_step_all], dcgs_all[best_step_all]])) >= 0:
              best_step_all = step // 1000
              torch.save(policy_network.state_dict(), params['log_w_features_dir'] + 'best_policy_net.pth')
              torch.save(value_network.state_dict(), params['log_w_features_dir'] + 'best_value_net.pth')
              torch.save(state_representation.state_dict(), params['log_w_features_dir'] + 'best_state_repr.pth')

      step += 1

  0%|          | 0/52 [00:00<?, ?it/s]

user = 82


  2%|▏         | 1/52 [00:13<11:14, 13.22s/it]

user = 179


  4%|▍         | 2/52 [00:26<10:58, 13.16s/it]

...

 96%|█████████▌| 50/52 [40:36<00:50, 25.18s/it]

user = 203


 98%|█████████▊| 51/52 [45:19<01:42, 102.42s/it]

user = 243


100%|██████████| 52/52 [45:27<00:00, 52.44s/it]


In [ ]:
# save policy, q-value and state
torch.save(policy_network.state_dict(), params['log_w_features_dir'] + 'policy_net_final.pth')
torch.save(value_network.state_dict(), params['log_w_features_dir'] + 'value_net_final.pth')
torch.save(state_representation.state_dict(), params['log_w_features_dir'] + 'state_repr_final.pth')

In [ ]:
# save memory
with open(params['log_w_features_dir'] + 'memory.pickle', 'wb') as f:
  pickle.dump(train_env.memory, f)

#### Evaluation on test set

In [ ]:
with open(params['log_w_features_dir'] + 'memory.pickle', 'rb') as f:
    memory = pickle.load(f)

In [ ]:
# evaluate best policy on test set
state_rep = State_Representation_w_Features(num_of_countries, num_of_age_buckets, num_of_books, params['embedding_dim'], params['hidden_dim'])
policy = Actor(params['embedding_dim'], params['hidden_dim'])
state_rep.load_state_dict(torch.load(params['log_w_features_dir'] + 'best_state_repr.pth'))
policy.load_state_dict(torch.load(params['log_w_features_dir'] + 'best_policy_net.pth'))

<All keys matched successfully>

In [ ]:
hit, dcg = evaluation(policy, state_rep, memory, full_loader)
print('hit rate: ', hit, 'dcg: ', dcg)

hit rate:  0.9656041372825576 dcg:  0.47740640385509453


#### Inference

In [ ]:
random_user = test_data['user'].sample().iloc[0]
random_user

221

In [ ]:
temp_test_data = test_data[test_data.user == random_user]
temp = pd.merge(temp_test_data, filtered_df, on=['user', 'book'])
temp

,user,book,isbn,title,author,user_id,rating,best_seller
0,221,691,373218192,Engaging The Enemy,Nora Roberts,216012,7.0,True
1,221,1319,425192695,Girls of the Summer,Barbara Bretton,216012,7.0,False
2,221,1417,671864173,WAITING TO EXHALE,Terry McMillan,216012,9.0,True
3,221,1437,1551665638,Call It Destiny (Mira),Jayne Ann Krentz,216012,7.0,True
4,221,2590,373218036,"Truly, Madly Manhattan",Nora Roberts,216012,8.0,False
5,221,2891,373218958,Less Of A Stranger (Silhouette Single Title),Nora Roberts,216012,8.0,False
6,221,2959,373197187,Cattleman's Pride (Silhouette Romance),Diana Palmer,216012,8.0,True
7,221,3092,373218419,Blithe Images,Nora Roberts,216012,7.0,False
8,221,3257,373834349,Mail-Order Marriages (Harlequin Promo),Debbie Macomber,216012,8.0,False
9,221,4222,446610550,The Playboy,Carly Phillips,216012,7.0,False


In [ ]:
recommendations = []

for model, state_representation in zip([policy], [state_rep]):
    env = Environment(test_matrix)
    user, country, age, memory = env.reset(random_user)

    # give 2 books that the user has rated in the past
    user, memory, reward, _ = env.step(torch.tensor([3092]))
    user, memory, reward, _ = env.step(torch.tensor([4092]))

    # get next 3 recommendations based on the two books above
    recs = []
    for i in range(3):
        action_emb = model(state_representation(country, age, memory))
        action = model.get_action(
            user,
            torch.tensor(env.memory[user.detach().cpu().numpy().astype(int), :]),
            state_representation,
            action_emb,
            torch.tensor([item for item in env.available_books if item not in env.viewed_books]).long()
        )
        user, memory, reward, _ = env.step(action)
        recs.append(action)

    recommendations.append(recs)

In [ ]:
recos = [r.item() for r in recommendations[0]]
recos

[15653, 2959, 29394]

In [ ]:
recommended_books = filtered_df[filtered_df.book.isin(recos)][['title', 'author', 'best_seller']].drop_duplicates()
recommended_books

,title,author,best_seller
94446,Cattleman's Pride (Silhouette Romance),Diana Palmer,True
235903,The Arming of Europe and the Making of the Fir...,David G. Herrmann,True
348036,The Thousand-Mile War: World War II in Alaska ...,Brian Garfield,False
